#### Importing  Libraries

In [1]:
import json
import random

#### Structuring Data

In [2]:
class Sentiment:
    POSITIVE = "Positive"
    NEGATIVE = "Negative"
    

class Review:
    def __init__(self,text,score):
        self.text = text
        self.score = score
        self.sentiment = self.get_sentiment()
        
    def get_sentiment(self):
        if self.score < 3:
            return Sentiment.NEGATIVE
        else: # 3 and above
            return Sentiment.POSITIVE
        
# to balance the dataset      
class ReviewContainer:
    def __init__(self,reviews):
        self.reviews = reviews
    def get_text(self):
        return [x.text for x in self.reviews]
    
    def get_sentiment(self):
        return [x.sentiment for x in self.reviews]
        
    def evenly_distribute(self):
        negative = list(filter(lambda x: x.sentiment == Sentiment.NEGATIVE, self.reviews))
        positive = list(filter(lambda x: x.sentiment == Sentiment.POSITIVE, self.reviews))
        reduced_positive = positive[:len(negative)]
        self.reviews = negative + reduced_positive
        random.shuffle(self.reviews)
        

#### Loading Data

In [4]:

filename = './Books_small_10000.json'
review_data = []
with open(filename) as f:
    for line in f:
        review = json.loads(line)
        review_data.append(Review(review["reviewText"],review["overall"]))
review_data[5].sentiment 

'Positive'

#### Data Preparation

In [5]:
len(review_data)

10000

In [6]:
from sklearn.model_selection import train_test_split
train,test = train_test_split(review_data,test_size=0.33,random_state=42)

train_container = ReviewContainer(train)
test_container = ReviewContainer(test)

In [7]:
print(train[0].text)
print(test[0].sentiment)

Olivia Hampton arrives at the Dunraven family home as cataloger of their extensive library. What she doesn't expect is a broken carriage wheel on the way. Nor a young girl whose mind is clearly gone, an old man in need of care himself (and doesn&#8217;t quite seem all there in Olivia&#8217;s opinion). Furthermore, Marion Dunraven, the only sane one of the bunch and the one Olivia is inexplicable drawn to, seems captive to everyone in the dusty old house. More importantly, she doesn't expect to fall in love with Dunraven's daughter Marion.Can Olivia truly believe the stories of sadness and death that surround the house, or are they all just local neighborhood rumor?Was that carriage trouble just a coincidence or a supernatural sign to stay away? If she remains, will the Castle&#8217;s dark shadows take Olivia down with them or will she and Marion long enough to declare their love?Patty G. Henderson has created an atmospheric and intriguing story in her Gothic tale. I found this to be an

In [8]:
# Breaking data down into features and labels
train_container.evenly_distribute()
train_x = train_container.get_text()
train_y = train_container.get_sentiment()

test_container.evenly_distribute()
test_x = test_container.get_text()
test_y = test_container.get_sentiment()

print(train_y.count(Sentiment.POSITIVE))
print(train_y.count(Sentiment.NEGATIVE))

436
436


#### Vectorization

In [9]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

vectorizer = TfidfVectorizer()
train_x_vectors = vectorizer.fit_transform(train_x)
print(train_x_vectors.shape)
test_x_vectors = vectorizer.transform(test_x)
print(test_x_vectors.shape)


(872, 8900)
(416, 8900)


#### Support Vector Machines

#### Classification

In [10]:
from sklearn import svm

svm_classifier = svm.SVC(kernel ='linear')

svm_classifier.fit(train_x_vectors,train_y)

svm_classifier.predict(test_x_vectors[0])

array(['Negative'], dtype='<U8')

#### Decision Tree


In [11]:
from sklearn.tree import DecisionTreeClassifier

tree_classifier = DecisionTreeClassifier()
tree_classifier.fit(train_x_vectors, train_y)

tree_classifier.predict(test_x_vectors[15])

array(['Negative'], dtype='<U8')

#### Naive Bayes

In [12]:
from sklearn.naive_bayes import GaussianNB

gaussian = DecisionTreeClassifier()
gaussian .fit(train_x_vectors, train_y)

gaussian.predict(test_x_vectors[0])

array(['Negative'], dtype='<U8')

#### Logistic Regression

In [13]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression()
log_reg.fit(train_x_vectors, train_y)

log_reg.predict(test_x_vectors[0])

array(['Negative'], dtype='<U8')

#### Evaluation

In [14]:
# accuracy
print(svm_classifier.score(test_x_vectors, test_y))
print(tree_classifier.score(test_x_vectors, test_y))
print(gaussian.score(test_x_vectors, test_y))
print(log_reg.score(test_x_vectors, test_y))

0.7716346153846154
0.6129807692307693
0.6322115384615384
0.7740384615384616


In [15]:
# F1 score
from sklearn.metrics import f1_score
f1_score(test_y, svm_classifier.predict(test_x_vectors), average=None, labels=[Sentiment.POSITIVE, Sentiment.NEGATIVE])
#f1_score(test_y, log_reg.predict(test_x_vectors), average=None, labels=[Sentiment.POSITIVE, Sentiment.NEGATIVE])

array([0.76658477, 0.77647059])

In [16]:
test_set = ['very fun', "bad book do not buy", 'horrible waste of time']
new_test = vectorizer.transform(test_set)

svm_classifier.predict(new_test)

array(['Positive', 'Negative', 'Negative'], dtype='<U8')

#### Model Tuning (Grid Search CV)

In [18]:
from sklearn.model_selection import GridSearchCV

parameters = {'kernel': ('linear', 'rbf'), 'C': (1,4,8,16,32)}

svc = svm.SVC()
tuned_clf = GridSearchCV(svc, parameters, cv=5)
tuned_clf.fit(train_x_vectors, train_y)

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': (1, 4, 8, 16, 32), 'kernel': ('linear', 'rbf')})

In [19]:
print(tuned_clf.score(test_x_vectors, test_y))

0.7884615384615384
